In [3]:
"""
This script imports necessary modules and sets up the environment for generating a Canvas assignment report.

It uses the canvasapi module to interact with the Canvas LMS.
"""

from canvasapi import Canvas
from config import *
import datetime
import csv
import tqdm
import pandas as pd

In [15]:
"""
This code retrieves submissions and rubric information for a specific assignment in Canvas.
"""

canvas = Canvas(CANVAS_URL, CANVAS_TOKEN)
course = canvas.get_course(course_id)
assignment = course.get_assignment(assignment_id)

submissions = [x for x in assignment.get_submissions(include=["user",
                                                              "submission_comments",
                                                              "rubric_assessment"]
                                                    )]
rubric = assignment.rubric

rubric_rating_headers = [f"RATING_{x['description']}" for x in rubric]
rubric_score_headers = [f"SCORE_{x['description']}" for x in rubric]

header_list = [
    "last_name",
    "first_name",
    "sis_user_id",
    "submitted_at",
    "seconds_late",
    "status",
    "posted_at",
    "score",
    "grader",
    "comments"]

header_list += rubric_rating_headers + rubric_score_headers

In [18]:
def get_rubric_rating(rubric, rubric_assessment):
    """
    Retrieves the descriptions of the ratings for each rubric item in the rubric assessment.

    Parameters:
    rubric (list): A list of rubric items.
    rubric_assessment (dict): A dictionary containing the rubric assessment data.

    Returns:
    list: A list of rating descriptions for each rubric item in the rubric assessment.
    """
    ratings_list = []
    rubric_flag = False
    for item in rubric:
        rating_id = rubric_assessment[item["id"]]["rating_id"]
        for ratings in item["ratings"]:
            if ratings["id"] == rating_id:
                if ratings["description"]:
                    ratings_list.append(ratings["description"])
                    rubric_flag = True
                else:
                    ratings_list.append("")
        if rubric_flag:
            rubric_flag = False
        else:
            ratings_list.append("")
            
    return ratings_list

def get_rubric_score(rubric, rubric_assessment):
    """
    Calculates the score for each rubric item based on the rubric assessment.

    Parameters:
    rubric (list): The rubric containing the criteria and ratings.
    rubric_assessment (dict): The rubric assessment containing the rating for each rubric item.

    Returns:
    list: A list of scores for each rubric item.
    """
    ratings_list = []
    for item in rubric:
        rating_id = rubric_assessment[item["id"]]["rating_id"]
        for ratings in item["ratings"]:
            if ratings["id"] == rating_id:
                    ratings_list.append(ratings["points"])
        
    return ratings_list

def build_submission_string(header_list, submission):
    """
    Builds a row of data for a submission in a Canvas assignment report.

    Args:
        submission (Submission): The submission object representing a student's submission.

    Returns:
        list: A list containing the row of data for the submission, including student information,
              submission details, grading information, and rubric ratings and scores.
    """
    
    sortable_name = f'{submission.user["sortable_name"]}'
    last_name, first_name = sortable_name.split(", ")
    sis_user_id = submission.user["sis_user_id"]
    submitted_at = submission.submitted_at
    seconds_late = submission.seconds_late
    status = submission.workflow_state
    posted_at = submission.posted_at
    score = submission.score

    try:
        grader = canvas.get_user(submission.grader_id).sortable_name
    except:
        grader = ""
    
    try:
        rubric_assessment = submission.rubric_assessment
    except:
        rubric_assessment = ""
        
    comments = ", ".join([f"{x["author_name"]} - {x["comment"]}" for x in submission.submission_comments])

    if rubric_assessment:
        rubric_rating = get_rubric_rating(rubric, rubric_assessment)
        rubric_score = get_rubric_score(rubric, rubric_assessment)
    else:
        rubric_rating = [""]*len(rubric)
        rubric_score = [""]*len(rubric)

    values = [
        last_name,
        first_name,
        sis_user_id,
        submitted_at,
        seconds_late,
        status,
        posted_at,
        score,
        grader,
        comments
    ]

    values += rubric_rating + rubric_score

    row = {}

    for header, value in zip(header_list, values):
        row[header] = value
        
    return row

In [19]:
import datetime
import tqdm

filename = f"{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}_{course_id}_{assignment_id}.xlsx"

data = []
for submission in tqdm.tqdm(submissions[:10], desc="Writing submissions to Excel file"):
    data.append(build_submission_string(header_list, submission))

df = pd.DataFrame(data)
df.to_excel(filename, index=False)


Writing submissions to Excel file: 100%|██████████| 10/10 [00:00<00:00, 15.16it/s]


In [135]:
data

[['last_name',
  'first_name',
  'sis_user_id',
  'submitted_at',
  'seconds_late',
  'status',
  'posted_at',
  'score',
  'grader',
  'RATING_Knowledge and Comprehension of Background - 30 %',
  'RATING_Handling of Literature - 20 %',
  'RATING_Aims and Approach - 20 %',
  'RATING_Structure and Organisation - 10 %',
  'RATING_Language - 10 %',
  'RATING_Presentation and Layout - 10 %',
  'RATING_Academic Integrity',
  'SCORE_Knowledge and Comprehension of Background - 30 %',
  'SCORE_Handling of Literature - 20 %',
  'SCORE_Aims and Approach - 20 %',
  'SCORE_Structure and Organisation - 10 %',
  'SCORE_Language - 10 %',
  'SCORE_Presentation and Layout - 10 %',
  'SCORE_Academic Integrity',
  'rubric_comments'],
 ['Abdulmalik',
  'Nadia',
  '201279226hlnharoo',
  None,
  4121415,
  'unsubmitted',
  None,
  None,
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 ['Bashagha',
  'Ali',
  '201428151hlabasha',
  '2024-01-08T23:48:16Z',
  0,

In [121]:
build_submission_string(submission)

TypeError: can only concatenate str (not "list") to str